✅ Full training pipeline for ARIMA + Stacking model
✅ Preprocessing pipeline using ColumnTransformer for reproducibility
✅ Saving ARIMA model, Stacking pipeline, and Hybrid weights
✅ Example inference script to load and predict

In [ ]:
# KL High-rise Property Price Forecast with Calibrated Dynamic Features
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
import joblib
import pickle

# -----------------------------
# STEP 1: Load MHPI_Annual.csv for ARIMA
# -----------------------------
mhpi_df = pd.read_csv("https://raw.githubusercontent.com/englian1123/KL-High-Rise-Data/refs/heads/main/MHPI_Annual.csv")
price_series = mhpi_df['Price']

# Fit ARIMA and get in-sample predictions
arima_model = ARIMA(price_series, order=(1, 1, 1)).fit()
arima_in_sample = arima_model.predict(start=1, end=len(price_series)-1)
arima_rmse = np.sqrt(mean_squared_error(price_series[1:], arima_in_sample))

# Forecast 2025–2030
arima_forecast = arima_model.forecast(steps=6)
forecast_years = list(range(2025, 2031))

# -----------------------------
# STEP 2: Load KLHighRise.csv for Stacking Model
# -----------------------------
df = pd.read_csv("https://raw.githubusercontent.com/englian1123/KL-High-Rise-Data/refs/heads/main/KLHighRise.csv")

# Basic preprocessing
df['ParcelArea'] = df['ParcelArea'].astype(str).str.extract(r'(\d+\.?\d*)')[0].astype(np.float32)
df['Tenure'] = df['Tenure'].map({'Freehold': 1, 'Leasehold': 0}).fillna(0).astype(np.float32)

price_cap = df['TransactionPrice'].quantile(0.90)
df['TransactionPrice'] = np.clip(df['TransactionPrice'], 0, price_cap).astype(np.float32)
area_cap = df['ParcelArea'].quantile(0.90)
df['ParcelArea'] = np.clip(df['ParcelArea'], 0, area_cap).astype(np.float32)

df['TransactionPrice'] = np.log1p(df['TransactionPrice']).astype(np.float32)
df['ParcelArea'] = np.log1p(df['ParcelArea']).astype(np.float32)

scheme_encoding = df.groupby('SchemeName')['TransactionPrice'].mean().astype(np.float32)
df['Scheme_Name_encoded'] = df['SchemeName'].map(scheme_encoding).fillna(scheme_encoding.mean()).astype(np.float32)

df = pd.get_dummies(df, columns=['Mukim'], drop_first=True, dtype=np.float32)

unit_level_map = {'03A': 4, '12B': 12, '13A': 14, '23A': 24, '33A': 34, '43A': 44, '53A': 54,
                  'B': 0, 'D': 0, 'G': 0, 'LG': 0, 'MZ': 0, 'P': 0, 'UG': 0}
df['UnitLevel_clean'] = df['UnitLevel'].replace(unit_level_map)
unit_level_mean = pd.to_numeric(df['UnitLevel_clean'], errors='coerce').mean()
df['UnitLevel_clean'] = pd.to_numeric(df['UnitLevel_clean'], errors='coerce').fillna(unit_level_mean).astype(np.float32)

selected_features = ['Scheme_Name_encoded', 'ParcelArea', 'Mukim_Mukim Batu', 'UnitLevel_clean', 'Tenure']
X = df[selected_features]
y = df['TransactionPrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------
# STEP 3: Build Stacking Model
# -----------------------------
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42, verbosity=0)),
    ('lgbm', LGBMRegressor(n_estimators=100, random_state=42))
]
meta_model = LinearRegression()

stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model, passthrough=True)
stacking_model.fit(X_train, y_train)

# Predictions on test set
y_pred_stack = stacking_model.predict(X_test)
y_pred_stack_orig = np.expm1(y_pred_stack)
y_test_orig = np.expm1(y_test)
stacking_rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_stack_orig))

# -----------------------------
# STEP 4: Compute Dynamic Weights
# -----------------------------
inv_arima = 1 / arima_rmse
inv_stack = 1 / stacking_rmse
weight_arima = inv_arima / (inv_arima + inv_stack)
weight_stack = 1 - weight_arima

print(f"ARIMA RMSE: {arima_rmse:.2f}, Stacking RMSE: {stacking_rmse:.2f}")
print(f"Weights -> ARIMA: {weight_arima:.3f}, Stacking: {weight_stack:.3f}")

# -----------------------------
# STEP 5: Save Models and Pipeline
# -----------------------------
# Save ARIMA model
with open("arima_model.pkl", "wb") as f:
    pickle.dump(arima_model, f)

# Save stacking model
joblib.dump(stacking_model, "stacking_model.pkl")

# Save weights
weights = {"weight_arima": weight_arima, "weight_stack": weight_stack}
with open("hybrid_weights.pkl", "wb") as f:
    pickle.dump(weights, f)

print("✅ Models and weights saved successfully!")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000819 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 514
[LightGBM] [Info] Number of data points in the train set: 11513, number of used features: 5
[LightGBM] [Info] Start training from score 13.333459
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 511
[LightGBM] [Info] Number of data points in the train set: 9210, number of used features: 5
[LightGBM] [Info] Start training from score 13.334137
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Inf

# Inference Script

In [ ]:
import pickle
import joblib
import numpy as np
import pandas as pd

# Load ARIMA model
with open("arima_model.pkl", "rb") as f:
    arima_loaded = pickle.load(f)

# Load stacking model
stacking_loaded = joblib.load("stacking_model.pkl")

# Load weights
with open("hybrid_weights.pkl", "rb") as f:
    weights_loaded = pickle.load(f)

# Example new data for stacking model
new_data = pd.DataFrame({
    'Scheme_Name_encoded': [5.2],
    'ParcelArea': [2.8],
    'Mukim_Mukim Batu': [0],
    'UnitLevel_clean': [12],
    'Tenure': [1]
})

# Predict stacking price (original scale)
stack_pred = np.expm1(stacking_loaded.predict(new_data))

# Predict ARIMA next step
arima_pred = arima_loaded.forecast(steps=1).iloc[0]

# Hybrid forecast
hybrid_pred = (weights_loaded['weight_arima'] * arima_pred) + (weights_loaded['weight_stack'] * stack_pred[0])
print("Hybrid Forecast:", hybrid_pred)

Hybrid Forecast: 513033.82617268054


# Full Training + Save Pipeline
✅ Full training pipeline for ARIMA + Stacking model

✅ Preprocessing pipeline using ColumnTransformer for raw input transformation

✅ Saving ARIMA model, Stacking pipeline, and Hybrid weights

✅ Example inference script for **raw user input**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
import joblib
import pickle

# -----------------------------
# STEP 1: Load MHPI_Annual.csv for ARIMA
# -----------------------------
mhpi_df = pd.read_csv("https://raw.githubusercontent.com/englian1123/KL-High-Rise-Data/refs/heads/main/MHPI_Annual.csv")
price_series = mhpi_df['Price']

# Fit ARIMA and compute RMSE
arima_model = ARIMA(price_series, order=(1, 1, 1)).fit()
arima_in_sample = arima_model.predict(start=1, end=len(price_series)-1)
arima_rmse = np.sqrt(mean_squared_error(price_series[1:], arima_in_sample))

# Forecast 2025–2030
arima_forecast = arima_model.forecast(steps=6)
forecast_years = list(range(2025, 2031))

# -----------------------------
# STEP 2: Load KLHighRise.csv for Stacking Model
# -----------------------------
df = pd.read_csv("https://raw.githubusercontent.com/englian1123/KL-High-Rise-Data/refs/heads/main/KLHighRise.csv")

# Compute target encoding for SchemeName
scheme_encoding = df.groupby('SchemeName')['TransactionPrice'].mean().astype(np.float32)

# -----------------------------
# STEP 3: Define Custom Transformers
# -----------------------------
unit_level_map = {'03A': 4, '12B': 12, '13A': 14, '23A': 24, '33A': 34, '43A': 44, '53A': 54,
                  'B': 0, 'D': 0, 'G': 0, 'LG': 0, 'MZ': 0, 'P': 0, 'UG': 0}

def clean_unit_level(x):
    # Apply replace and then infer objects to handle the FutureWarning explicitly
    cleaned_df = pd.DataFrame(x).replace(unit_level_map).infer_objects(copy=False)
    return cleaned_df.apply(pd.to_numeric, errors='coerce').fillna(0)

def encode_scheme(x):
    # x is expected to be a DataFrame with the 'SchemeName' column
    encoded_series = x['SchemeName'].map(scheme_encoding).fillna(scheme_encoding.mean())
    return encoded_series.to_frame(name='Scheme_Name_encoded') # Ensure 2D output

def log_transform(x):
    return np.log1p(x)

# -----------------------------
# STEP 4: Outlier Caps
# -----------------------------
price_cap = df['TransactionPrice'].quantile(0.90)
area_cap = df['ParcelArea'].astype(str).str.extract(r'(\d+\.?\d*)')[0].astype(float).quantile(0.90)

# Define a named function to cap outliers that captures area_cap for pickling compatibility
def cap_parcel_area(x):
    return np.clip(x, 0, area_cap)

# -----------------------------
# STEP 5: Build Preprocessing Pipeline
# -----------------------------
preprocessor = ColumnTransformer(transformers=[
    ('scheme', FunctionTransformer(encode_scheme), ['SchemeName']),
    ('parcel', Pipeline(steps=[
        ('cap', FunctionTransformer(cap_parcel_area)), # Use the named function here instead of lambda
        ('log', FunctionTransformer(log_transform))
    ]), ['ParcelArea']),
    ('unit_clean', FunctionTransformer(clean_unit_level), ['UnitLevel']),
    ('tenure', 'passthrough', ['Tenure']),
    ('mukim', OneHotEncoder(drop='first', handle_unknown='ignore'), ['Mukim'])
])

# -----------------------------
# STEP 6: Prepare Data
# -----------------------------
df['Tenure'] = df['Tenure'].map({'Freehold': 1, 'Leasehold': 0}).fillna(0)
df['Mukim'] = df['Mukim'].fillna('Mukim Batu')
X = df[['SchemeName', 'ParcelArea', 'Mukim', 'UnitLevel', 'Tenure']]
y = np.log1p(np.clip(df['TransactionPrice'], 0, price_cap))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------
# STEP 7: Build Stacking Model Pipeline
# -----------------------------
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42, verbosity=0)),
    ('lgbm', LGBMRegressor(n_estimators=100, random_state=42))
]
meta_model = LinearRegression()

stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model, passthrough=True)

full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', stacking_model)
])

full_pipeline.fit(X_train, y_train)

# Predictions and RMSE
y_pred_stack = full_pipeline.predict(X_test)
stacking_rmse = np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred_stack)))

# -----------------------------
# STEP 8: Compute Dynamic Weights
# -----------------------------
inv_arima = 1 / arima_rmse
inv_stack = 1 / stacking_rmse
weight_arima = inv_arima / (inv_arima + inv_stack)
weight_stack = 1 - weight_arima

print(f"ARIMA RMSE: {arima_rmse:.2f}, Stacking RMSE: {stacking_rmse:.2f}")
print(f"Weights -> ARIMA: {weight_arima:.3f}, Stacking: {weight_stack:.3f}")

# -----------------------------
# STEP 9: Save Models and Pipeline
# -----------------------------
with open("arima_model.pkl", "wb") as f:
    pickle.dump(arima_model, f)

joblib.dump(full_pipeline, "stacking_pipeline.pkl")

weights = {"weight_arima": weight_arima, "weight_stack": weight_stack}
with open("hybrid_weights.pkl", "wb") as f:
    pickle.dump(weights, f)

print("✅ Models, pipeline, and weights saved successfully!")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003228 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 528
[LightGBM] [Info] Number of data points in the train set: 11513, number of used features: 11
[LightGBM] [Info] Start training from score 13.333459
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000639 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 523
[LightGBM] [Info] Number of data points in the train set: 9210, number of used features: 11
[LightGBM] [Info] Start training from score 13.334137
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000151 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bin

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000152 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 524
[LightGBM] [Info] Number of data points in the train set: 9210, number of used features: 11
[LightGBM] [Info] Start training from score 13.331412
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000563 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 525
[LightGBM] [Info] Number of data points in the train set: 9211, number of used features: 11
[LightGBM] [Info] Start training from score 13.336253


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000152 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 527
[LightGBM] [Info] Number of data points in the train set: 9211, number of used features: 11
[LightGBM] [Info] Start training from score 13.331686


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


ARIMA RMSE: 12485.56, Stacking RMSE: 82479.44
Weights -> ARIMA: 0.869, Stacking: 0.131
✅ Models, pipeline, and weights saved successfully!


# Inference Script - Raw Input

In [ ]:
import pickle
import joblib
import numpy as np
import pandas as pd

# -----------------------------
# Load Saved Assets
# -----------------------------
with open("arima_model.pkl", "rb") as f:
    arima_loaded = pickle.load(f)

stacking_pipeline_loaded = joblib.load("stacking_pipeline.pkl")

with open("hybrid_weights.pkl", "rb") as f:
    weights_loaded = pickle.load(f)

# -----------------------------
# Example Raw User Input
# -----------------------------
raw_input = pd.DataFrame({
    'SchemeName': ['FERNLEA COURT'],   # Raw scheme name
    'ParcelArea': [1200],           # Raw parcel area in sq ft
    'Mukim': ['Mukim Batu'],        # Raw Mukim name
    'UnitLevel': ['12B'],           # Raw unit level
    'Tenure': [1]                   # Freehold (1), Leasehold (0)
})

# -----------------------------
# Predict Using Stacking Pipeline
# -----------------------------
stack_pred = np.expm1(stacking_pipeline_loaded.predict(raw_input))  # Convert back to original scale

# -----------------------------
# Predict Using ARIMA
# -----------------------------
arima_pred = arima_loaded.forecast(steps=1).iloc[0]

# -----------------------------
# Compute Hybrid Forecast
# -----------------------------
hybrid_pred = (weights_loaded['weight_arima'] * arima_pred) + (weights_loaded['weight_stack'] * stack_pred[0])

print("Stacking Prediction:", stack_pred[0])
print("ARIMA Prediction:", arima_pred)
print("Hybrid Forecast:", hybrid_pred)

Stacking Prediction: 1474134.6830461181
ARIMA Prediction: 583491.828471573
Hybrid Forecast: 700589.412364786


/tmp/ipython-input-701715242.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_df = pd.DataFrame(x).replace(unit_level_map).infer_objects(copy=False)
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
import pickle
import joblib
import numpy as np
import pandas as pd

# Load saved assets
with open("arima_model.pkl", "rb") as f:
    arima_loaded = pickle.load(f)

stacking_pipeline_loaded = joblib.load("stacking_pipeline.pkl")

with open("hybrid_weights.pkl", "rb") as f:
    weights_loaded = pickle.load(f)

# Load historical data to get last year
mhpi_df = pd.read_csv("https://raw.githubusercontent.com/englian1123/KL-High-Rise-Data/refs/heads/main/MHPI_Annual.csv")
last_year = int(mhpi_df['Year of Year'].iloc[-1])

# --- User Input ---
desired_year = 2028  # Change this as needed

# Validate desired_year
if not isinstance(desired_year, int):
    raise ValueError("Desired year must be an integer.")

steps_ahead = desired_year - last_year

# Handle cases where desired_year is not in the future
if steps_ahead <= 0:
    print(f"Desired year {desired_year} is not after the last historical year {last_year}.")
    if desired_year == last_year:
        print(f"Returning the last observed price for {last_year} as the 'forecast'.")
        arima_pred = mhpi_df['Price'].iloc[-1]  # Use last actual price if desired_year is current year
    else:  # desired_year < last_year
        raise ValueError(f"Cannot forecast for a past year ({desired_year}). Last historical year is {last_year}.")
else:
    # ARIMA forecast for desired year
    forecast_values = arima_loaded.forecast(steps=steps_ahead)
    arima_pred = forecast_values.iloc[-1] if hasattr(forecast_values, 'iloc') else forecast_values[-1]

# Example raw input for stacking model
raw_input = pd.DataFrame({
    'SchemeName': ['FABER INDAH'],
    'ParcelArea': [1200],
    'Mukim': ['Mukim Batu'],
    'UnitLevel': ['12B'],
    'Tenure': [1]
})

# Predict stacking price (original scale)
stack_pred = np.expm1(stacking_pipeline_loaded.predict(raw_input))

# Hybrid forecast
hybrid_pred = (weights_loaded['weight_arima'] * arima_pred) + (weights_loaded['weight_stack'] * stack_pred[0])

# Output
print(f"Forecast for {desired_year}:")
print("ARIMA Prediction:", arima_pred)
print("Stacking Prediction:", stack_pred[0])
print("Hybrid Forecast:", hybrid_pred)

Forecast for 2028:
ARIMA Prediction: 609707.0444733249
Stacking Prediction: 559188.2407966102
Hybrid Forecast: 603065.0669159518


/tmp/ipython-input-701715242.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_df = pd.DataFrame(x).replace(unit_level_map).infer_objects(copy=False)
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
!pip install fastapi uvicorn nest_asyncio pyngrok pandas numpy joblib

# Run FastAPI on Google Colab

In [ ]:
!pip install fastapi uvicorn nest_asyncio pyngrok pandas numpy joblib

import pickle
import joblib
import numpy as np
import pandas as pd
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import threading # Import threading module

import warnings
warnings.filterwarnings("ignore")

# ngrok authentication token
ngrok.set_auth_token("35T7iSFl2TobTiUGSAHkKsDqFgs_6UTj4KWQ9zfaYwTAin5GZ")

# Apply nest_asyncio for Colab
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI(title="KL High-Rise Price Forecast API")

# Load saved assets
with open("arima_model.pkl", "rb") as f:
    arima_loaded = pickle.load(f)

stacking_pipeline_loaded = joblib.load("stacking_pipeline.pkl")

with open("hybrid_weights.pkl", "rb") as f:
    weights_loaded = pickle.load(f)

# Load historical data
mhpi_df = pd.read_csv("https://raw.githubusercontent.com/englian1123/KL-High-Rise-Data/refs/heads/main/MHPI_Annual.csv")
last_year = int(mhpi_df['Year of Year'].iloc[-1])

# Request schema
class ForecastRequest(BaseModel):
    desired_year: int
    SchemeName: str
    ParcelArea: float
    Mukim: str
    UnitLevel: str
    Tenure: int

@app.post("/forecast")
def forecast_price(request: ForecastRequest):
    desired_year = request.desired_year
    steps_ahead = desired_year - last_year

    if steps_ahead <= 0:
        if desired_year == last_year:
            arima_pred = mhpi_df['Price'].iloc[-1]
        else:
            raise HTTPException(status_code=400, detail=f"Cannot forecast for past year {desired_year}. Last historical year is {last_year}.")
    else:
        forecast_values = arima_loaded.forecast(steps=steps_ahead)
        arima_pred = forecast_values.iloc[-1] if hasattr(forecast_values, 'iloc') else forecast_values[-1]

    raw_input = pd.DataFrame({
        'SchemeName': [request.SchemeName],
        'ParcelArea': [request.ParcelArea],
        'Mukim': [request.Mukim],
        'UnitLevel': [request.UnitLevel],
        'Tenure': [request.Tenure]
    })

    stack_pred = np.expm1(stacking_pipeline_loaded.predict(raw_input))
    hybrid_pred = (weights_loaded['weight_arima'] * arima_pred) + (weights_loaded['weight_stack'] * stack_pred[0])

    return {
        "desired_year": desired_year,
        "last_year": last_year,
        "ARIMA_Prediction": float(arima_pred),
        "Stacking_Prediction": float(stack_pred[0]),
        "Hybrid_Forecast": float(hybrid_pred)
    }

# Expose the app using ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Function to run uvicorn in a separate thread
def run_uvicorn(app, port):
    uvicorn.run(app, host="0.0.0.0", port=port)

# Create and start the thread
uvicorn_thread = threading.Thread(target=run_uvicorn, args=(app, 8000))
uvicorn_thread.start()

Public URL: NgrokTunnel: "https://nonsectorial-cami-bankerly.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [1569]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [ ]:
%%bash
curl -X POST "https://nonsectorial-cami-bankerly.ngrok-free.dev/forecast" \
-H "Content-Type: application/json" \
-d '{
      "desired_year": 2028,
      "SchemeName": "FERNLEA COURT",
      "ParcelArea": 1200,
      "Mukim": "Mukim Batu",
      "UnitLevel": "12B",
      "Tenure": 1
    }'

<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2642  100  2471  100   171  28601   1979 --:--:-- --:--:-- --:--:-- 30720


In [ ]:
from pyngrok import ngrok
ngrok.kill()
print("✅ ngrok tunnel stopped.")

✅ ngrok tunnel stopped.
